In [4]:
##do_runcode
##%overwritefile
##%file:src/do_dart_runcode.py
##%noruncode
    def do_runcode(self,return_code,fil_ename,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=return_code
        fil_ename=fil_ename
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''

        ##代码运行前
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,1)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        self._logln("The process :"+fil_ename)
        p = self.create_jupyter_subprocess(['dart','--verbose',fil_ename]+ magics['args'],cwd=None,shell=False,env=self.addkey2dict(magics,'env'))
        #p = self.create_jupyter_subprocess([binary_file.name]+ magics['args'],cwd=None,shell=False)
        #p = self.create_jupyter_subprocess([self.master_path, binary_file.name] + magics['args'],cwd='/tmp',shell=True)
        self.g_rtsps[str(p.pid)]=p
        return_code=p.returncode
        ##代码启动后
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
         
        if magics!=None and len(self.addkey2dict(magics,'showpid'))>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        while p.poll() is None:
            p.write_contents(magics)

        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()
        del self.g_rtsps[str(p.pid)]
        p.write_contents(magics)

        return_code=p.returncode
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,3)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        ##代码运行结束
        self.cleanup_files()
        if p.returncode != 0:
            self._log("Executable exited with code {}".format(p.returncode),2)
        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Dart/src/do_dart_runcode.py created successfully


In [5]:
##do_dart_create_codefile
##%overwritefile
##%file:src/do_dart_create_codefile.py
##%noruncode
    def do_create_codefile(self,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        fil_ename=''
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''

        ##调生成文件前用接口
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,1,1)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        
        with self.new_temp_file(suffix='.dart',dir=os.path.abspath('')) as source_file:
            source_file.write(code)
            source_file.flush()
            newsrcfilename=source_file.name
            fil_ename=newsrcfilename
            return_code=True
            # Generate new src file
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,1,2)
            if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
            if len(self.addkey2dict(magics,'file'))>0:
                    fil_ename=magics['file'][0]
            else: fil_ename=source_file.name
            # Generate executable file :being
            # Generate executable file :end
        if len(self.addkey2dict(magics,'noruncode'))>0:
            bcancel_exec=True
            retinfo= {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        
        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Dart/src/do_dart_create_codefile.py created successfully


In [6]:
##do_dart_preexecute
##%overwritefile
##%file:src/do_dart_preexecute.py
##%noruncode
    def do_preexecute(self,code, magics,silent, store_history=True,
                user_expressions=None, allow_stdin=False):
        bcancel_exec=False
        retinfo=self.get_retinfo()
        # magics, code = self.mag.filter(code)
        if len(self.addkey2dict(magics,'replcmdmode'))>0:
            bcancel_exec=True
            retinfo= self.send_replcmd(code, silent, store_history=True,
                user_expressions=None, allow_stdin=False)
            return bcancel_exec,retinfo,magics, code
        if (len(self.addkey2dict(magics,'noruncode'))>0
            and ( len(self.addkey2dict(magics,'command'))>0 
            or len(self.addkey2dict(magics,'dartcmd'))>0)):
            return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
        
        if len(self.addkey2dict(magics,'file'))<1:
            magics, code = self._add_main(magics, code)
        return_code=0
        fil_ename=''
        return bcancel_exec,retinfo,magics, code

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Dart/src/do_dart_preexecute.py created successfully


In [7]:
##do_dart_command
##%overwritefile
##%file:src/dart_do_dart_command.py
##%noruncode
    def do_dart_command(self,commands=None,cwd=None,magics=None):
        p = self.create_jupyter_subprocess(['dart']+commands,cwd=os.path.abspath(''),shell=False)
        self.g_rtsps[str(p.pid)]=p
        if magics!=None and len(magics['showpid'])>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        while p.poll() is None:
            p.write_contents()
        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()
        del self.g_rtsps[str(p.pid)]
        p.write_contents()

        if p.returncode != 0:
            self._write_to_stderr("[Dart kernel] Executable exited with code {}".format(p.returncode))
        else:
            self._write_to_stdout("[Dart kernel] Info:dart command success.")
        return


[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Dart/src/dart_do_dart_command.py created successfully


In [8]:
##do_flutter_command
##%overwritefile
##%file:src/dart_do_flutter_command.py
##%noruncode
    def do_flutter_command(self,commands=None,cwd=None,magics=None):
        p = self.create_jupyter_subprocess(['flutter']+commands,cwd=os.path.abspath(''),shell=False)
        self.g_rtsps[str(p.pid)]=p
        if magics!=None and len(magics['showpid'])>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        while p.poll() is None:
            p.write_contents()
        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()
        del self.g_rtsps[str(p.pid)]
        p.write_contents()

        if p.returncode != 0:
            self._write_to_stderr("[Dart kernel] Executable exited with code {}".format(p.returncode))
        else:
            self._write_to_stdout("[Dart kernel] Info:flutter command success.")
        return


[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Dart/src/dart_do_flutter_command.py created successfully


In [4]:
## new kernel.py
##%overwritefile
##%file:../../../jupyter-MyDart-kernel/jupyter_MyDart_kernel/kernel.py
###%file:dart_kernel.py
##%noruncode
#
#   MyDart Jupyter Kernel
#   generated by MyPython
#
##%include:../../src/head.py
##%include:../../src/common.py
##%include:../../src/Mykernel.py
class DartKernel(MyKernel):
    implementation = 'jupyter_MyDart_kernel'
    implementation_version = '1.0'
    language = 'Dart'
    language_version = '2.X.X'
    language_info = {'name': 'text/x-csrc',
                     'mimetype': 'text/x-csrc',
                     'file_extension': '.dart'}
    runfiletype='script'
    banner = "Dart kernel.\n" \
             "Uses Dart, compiles in dart, and creates source code files and executables in temporary folder.\n"

    main_head = "\n" \
            "\n" \
            "int main(List<String> arguments){\n"

    main_foot = "\nreturn 0;\n}"
    
##//%include:../../src/comm_attribute.py

    def __init__(self, *args, **kwargs):
        super(DartKernel, self).__init__(*args, **kwargs)
        self.kernelinfo='[MyDart Kernel]'
        self._allow_stdin = True
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
#################
##%include:src/dart_do_dart_command.py
##%include:src/dart_do_flutter_command.py

##%include:src/do_dart_runcode.py
##%include:src/do_dart_create_codefile.py
##%include:src/do_dart_preexecute.py



[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Dart/../../../jupyter-MyDart-kernel/jupyter_MyDart_kernel/kernel.py created successfully
